In [1]:
# from pyspark import SparkContext, SparkConf
# import pandas as pd

# # Initialize Spark Context
# conf = SparkConf().setAppName("Trust Rank Computation")
# sc = SparkContext(conf=conf)

# # Load data set into graph
# invoice_data = pd.read_csv("iron_dealers_data.csv")
# edges = invoice_data[['Seller ID', 'Buyer ID', 'Value']].values.tolist()
# graph = sc.parallelize(edges).map(lambda x: (x[0], x[1], float(x[2]))).groupByKey()

# # Initialize trust rank of each node
# default_trust_rank = 0.5
# node_trust_rank = graph.map(lambda x: (x[0], default_trust_rank))

# # Load bad nodes
# bad_nodes = pd.read_csv("bad.csv")
# bad_nodes_list = bad_nodes['Bad Id'].tolist()

# # Pregel Computation
# num_supersteps = 10

# def pregel_computation(vertex, messages):
#     trust_rank_sum = default_trust_rank
#     for message in messages:
#         trust_rank_sum += message
#     if vertex in bad_nodes_list:
#         new_trust_rank = 0
#     else:
#         new_trust_rank = 0.15 + 0.85 * trust_rank_sum
#     out_messages = []
#     for neighbor in vertex_neighbors[vertex]:
#         out_messages.append((neighbor, new_trust_rank / len(vertex_neighbors[vertex])))
#     return (new_trust_rank, out_messages)

# vertex_neighbors = graph.collectAsMap()

# for i in range(num_supersteps):
#     node_trust_rank = node_trust_rank.join(graph).flatMap(lambda x: pregel_computation(x[0], x[1][1])).reduceByKey(lambda x,y: x+y).mapValues(lambda x: 0.15 + 0.85 * x)

# # Output final trust rank
# node_trust_rank.saveAsTextFile("rank.txt")


ModuleNotFoundError: No module named 'pyspark'

In [2]:
import networkx as nx
import csv

# Load data set into graph
graph = nx.DiGraph()
with open("iron_dealers_data.csv") as f:
    reader = csv.reader(f)
    next(reader) # skip header
    for row in reader:
        seller_id, buyer_id, value = row
        graph.add_edge(seller_id, buyer_id, weight=float(value))

# Initialize trust rank of each node
default_trust_rank = 0.5
node_trust_rank = {node: default_trust_rank for node in graph.nodes()}

# Load bad nodes
bad_nodes = set()
with open("bad.csv") as f:
    reader = csv.reader(f)
    next(reader) # skip header
    for row in reader:
        bad_nodes.add(row[0])

# Trust Rank Computation
num_iterations = 10

for i in range(num_iterations):
    new_node_trust_rank = {}
    for node in graph.nodes():
        trust_rank_sum = default_trust_rank
        for neighbor in graph.predecessors(node):
            neighbor_trust_rank = node_trust_rank[neighbor]
            neighbor_out_degree = graph.out_degree(neighbor, weight='weight')
            trust_rank_sum += neighbor_trust_rank * graph[neighbor][node]['weight'] / neighbor_out_degree
        if node in bad_nodes:
            new_trust_rank = 0
        else:
            new_trust_rank = 0.15 + 0.85 * trust_rank_sum
        new_node_trust_rank[node] = new_trust_rank
    node_trust_rank = new_node_trust_rank

# Output final trust rank
with open("rank.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(['node_id', 'trust_rank'])
    for node, trust_rank in node_trust_rank.items():
        writer.writerow([node, trust_rank])
